# A/B Testing the Udacity Website

In these exercises, we'll be analyzing data on user behavior from an experiment run by Udacity, the online education company. More specifically, we'll be looking at a test Udacity ran to improve the onboarding process on their site.

Udacity's test is an example of an "A/B" test, in which some portion of users visiting a website (or using an app) are randomly selected to see a new version of the site. An analyst can then compare the behavior of users who see a new website design to users seeing their normal website to estimate the effect of rolling out the proposed changes to all users. While this kind of experiment has it's own name in industry (A/B testing), to be clear it's just a randomized experiment, and so everything we've learned about potential outcomes and randomized experiments apply here. 

(Udacity has generously provides the data from this test under an Apache open-source license, and you can find their [original writeup here](https://www.kaggle.com/tammyrotem/ab-tests-with-python/notebook). If you're interested in learning more on A/B testing in particular, it seems only fair while we use their data to flag they have a full course on the subject [here](https://www.udacity.com/course/ab-testing--ud257).)

## Udacity's Test

The test [is described by Udacity as follows](https://www.kaggle.com/tammyrotem/ab-tests-with-python/notebook): 

At the time of this experiment, Udacity courses currently have two options on the course overview page: "start free trial", and "access course materials".

**Current Conditions Before Change**

- If the student clicks "start free trial", they will be asked to enter their credit card information, and then they will be enrolled in a free trial for the paid version of the course. After 14 days, they will automatically be charged unless they cancel first.
- If the student clicks "access course materials", they will be able to view the videos and take the quizzes for free, but they will not receive coaching support or a verified certificate, and they will not submit their final project for feedback.

**Description of Experimented Change**

- In the experiment, Udacity tested a change where if the student clicked "start free trial", they were asked how much time they had available to devote to the course.
- If the student indicated 5 or more hours per week, they would be taken through the checkout process as usual. If they indicated fewer than 5 hours per week, a message would appear indicating that Udacity courses usually require a greater time commitment for successful completion, and suggesting that the student might like to access the course materials for free.
- At this point, the student would have the option to continue enrolling in the free trial, or access the course materials for free instead. This [screenshot](images/udacity_checkyoureready.png) shows what the experiment looks like.

**Udacity's Hope is that...**:

> this might set clearer expectations for students upfront, thus reducing the number of frustrated students who left the free trial because they didn't have enough time -- without significantly reducing the number of students to continue past the free trial and eventually complete the course. If this hypothesis held true, Udacity could improve the overall student experience and improve coaches' capacity to support students who are likely to complete the course.



## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_abtesting.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex4_avg_oec",
    "ex5_avg_guardrail",
    "ex7_ttest_pvalue",
    "ex9_ttest_pvalue_clicks",
    "ex10_num_obs",
    "ex11_guard_ate",
    "ex11_guard_pvalue",
    "ex11_oec_ate",
    "ex11_oec_pvalue",
    "ex14_se_treatment",
}
```


### Submission Limits

Please remember that you are **only allowed FOUR submissions to the autograder.** Your last submission (if you submit 4 or fewer times), or your third submission (if you submit more than 4 times) will determine your grade Submissions that error out will **not** count against this total.

That's one more than usual in case there are issues with exercise clarity.

In [26]:
import pandas as pd
import numpy as np
import warnings


warnings.filterwarnings("ignore")

pd.set_option("mode.copy_on_write", True)

## Import the Data

### Exercise 1

Begin by importing Udacity's data on user behavior [here.](https://github.com/nickeubank/MIDS_Data/tree/master/udacity_AB_testing) 

There are TWO datasets for this test — one for the control data (users who saw the original design), and one for treatment data (users who saw the experimental design). Udacity decided to show their test site to 1/2 of visitors, so there are roughly the same number of users appearing in each dataset (though this is not a requirement of AB tests).

Please remember to load the data directly from github to assist the autograder.

In [27]:
control_dataset = pd.read_csv(
    "https://github.com/nickeubank/MIDS_Data/raw/master/udacity_AB_testing/control_data.csv"
)

experiment_dataset = pd.read_csv(
    "https://github.com/nickeubank/MIDS_Data/raw/master/udacity_AB_testing/experiment_data.csv"
)

In [28]:
control_dataset

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0
5,"Thu, Oct 16",9670,823,138.0,82.0
6,"Fri, Oct 17",9008,748,146.0,76.0
7,"Sat, Oct 18",7434,632,110.0,70.0
8,"Sun, Oct 19",8459,691,131.0,60.0
9,"Mon, Oct 20",10667,861,165.0,97.0


### Exercise 2

Explore the data. Can you identify the unit of observation of the data (e.g. what is represented by each row)?

(Note this is not the only way that A/B test data can be collected and/or reported — this is just what Udacity provided, presumably to help address privacy concerns.)

> The unit of observation in the dataset is the total number of user engagement perday.

## Pick your measures

### Exercise 3

The easiest way to analyze this data is to stack it into a single dataset where each observation is a day-treatment-arm (so you should end up with two rows per day, one for those who are in the treated groups, and one for those who were in the control group). Note that currently nothing in the data identifies whether a given observation is a treatment group observation or a control group observation, so you'll want to make sure to add a "treatment" indicator variable.

The variables in the data are:

- Pageviews: number of unique users visiting homepage
- Clicks: number of those users clicking "Start Free Trial"
- Enrollments: Number of people enrolling in trial
- Payments: Number of people who eventually pay for the service. Note the `payment` column reports payments for the users who first visited the site on the reported date, not payments occurring on the reported date.

In [29]:
control_dataset["group"] = "control"
experiment_dataset["group"] = "treatment"

Dataset = pd.concat([control_dataset, experiment_dataset])
Dataset = Dataset.sort_values(by=["Date"])
Dataset

,Date,Pageviews,Clicks,Enrollments,Payments,group
34,"Fri, Nov 14",9192,735,NaN,NaN,control
34,"Fri, Nov 14",9304,770,NaN,NaN,treatment
27,"Fri, Nov 7",9424,781,NaN,NaN,control
27,"Fri, Nov 7",9272,767,NaN,NaN,treatment
6,"Fri, Oct 17",9088,780,127.0,44.0,treatment
...,...,...,...,...,...,...
4,"Wed, Oct 15",9793,832,140.0,94.0,treatment
11,"Wed, Oct 22",9947,838,162.0,92.0,control
11,"Wed, Oct 22",9737,801,128.0,70.0,treatment
18,"Wed, Oct 29",9262,727,201.0,96.0,treatment


### Exercise 4

Given Udacity's goals, what outcome are they hoping will be impacted by their manipulation?

Or, to ask the same question in the language of the Potential Outcomes Framework, what is their $Y$?

Or to ask the same question in the language of Kohavi, Tang and Xu, what is their *Overall Evaluation Criterion (OEC)*?

(I'm only asking one question, I'm just trying to phrase it using different terminologies we've encountered to help you see how they all fit together)

When you feel like you have your answer, please compute it. Store the average value of the variable in `results` under the key `ex4_avg_oec`. **Please round your answer to 4 decimal places.**

NOTE: You'll probably notice you have two choices to make when it comes to actually computing the OEC. 

- You could probably imagine either computing a ratio or a difference of two things — please calculate the difference.
- You may also be unsure whether to normalize by `Clicks`. Normalizing by clicks will help account for variation that comes from day-to-day variation in users, so it's a good thing to do. With infinite data, you'd expect to get the same results without normalizing by `Clicks` (since on average the same share of users are in each arm of the experiment), but for finite data it's a good strategy. Note that this is only ok because users make the choice to click or not *before* they see different versions of the website (it is "pre-treatment").

Just to make sure you're on track, your measure should have an average value of *about* 9%.

>The *Overall Evaluation Criterion (OEC)* is the **enrollment rate**, or **share of people clicking on** *start free trial* **who actually enrolled for the trial**, ***minus*** **payment rate**, the **share of people of clicked on** *start free trial* who **continue on the paied service after the trial**.

In [30]:
results = {}

In [31]:
Dataset["Enrollment Rate"] = Dataset["Enrollments"] / Dataset["Clicks"]

# calculate payment rate among clicked
Dataset["PaymentAmongClicked"] = Dataset["Payments"] / Dataset["Clicks"]


# calculate difference
Dataset["Enroll_Pay"] = Dataset["Enrollment Rate"] - Dataset["PaymentAmongClicked"]
Dataset

,Date,Pageviews,Clicks,Enrollments,Payments,group,Enrollment Rate,PaymentAmongClicked,Enroll_Pay
34,"Fri, Nov 14",9192,735,NaN,NaN,control,NaN,NaN,NaN
34,"Fri, Nov 14",9304,770,NaN,NaN,treatment,NaN,NaN,NaN
27,"Fri, Nov 7",9424,781,NaN,NaN,control,NaN,NaN,NaN
27,"Fri, Nov 7",9272,767,NaN,NaN,treatment,NaN,NaN,NaN
6,"Fri, Oct 17",9088,780,127.0,44.0,treatment,0.162821,0.056410,0.106410
...,...,...,...,...,...,...,...,...,...
4,"Wed, Oct 15",9793,832,140.0,94.0,treatment,0.168269,0.112981,0.055288
11,"Wed, Oct 22",9947,838,162.0,92.0,control,0.193317,0.109785,0.083532
11,"Wed, Oct 22",9737,801,128.0,70.0,treatment,0.159800,0.087391,0.072409
18,"Wed, Oct 29",9262,727,201.0,96.0,treatment,0.276479,0.132050,0.144429


In [32]:
Enroll_group = Dataset.groupby(["group"], as_index=False)["Enroll_Pay"].mean()
avg_oec = Enroll_group["Enroll_Pay"].mean()

ex4_avg_oec = round(avg_oec, 4)
results["ex4_avg_oec"] = ex4_avg_oec

In [33]:
results

{'ex4_avg_oec': 0.0941}

### Exercise 5

Given Udacity's goals, what outcome are they hoping will *not* be impacted by their manipulation? In other words, what do they want to measure to ensure their treatment doesn't have unintended negative consequences that might be really costly to their operation?

Note that while this isn't how Kohavi, Tang, and Xu use the term "guardrail metrics" — they usually use the term to refer to things we measure to ensure the experiment is working the way it should — some people would also use the term "guardrail metrics" for something that could be impacted even if the experiment is working correctly, but which the organization wants to track to ensure they aren't impacted because they are deemed really important.

Again, please normalize by `Clicks`. Store the average value of this guardrail metric as `ex5_avg_guardrail` and **round your answer to 4 decimal places.**

> The enrollment rate in the control group. This metric is crucial for Udacity because it represents the baseline performance of their platform and courses.If the treatment has unintended negative consequences that significantly reduce the enrollment rate in the control group, it could indicate potential issues with the manipulation or changes introduced by Udacity, which would be costly to their operation.



In [34]:
# Dataset["PaymentAmongClicked"] = Dataset["Payments"] / Dataset["Clicks"]

guard_group = Dataset.groupby(["group"], as_index=False)["PaymentAmongClicked"].mean()
guard_group["PaymentAmongClicked"].mean()

0.11582100814334004

In [35]:
ex5_avg_guardrail = guard_group["PaymentAmongClicked"].mean()
results["ex5_avg_guardrail"] = round(ex5_avg_guardrail, 4)
results

{'ex4_avg_oec': 0.0941, 'ex5_avg_guardrail': 0.1158}

## Validating The Data

### Exercise 6

Whenever you are working with experimental data, the first thing you want to do is verify that users actually were randomly sorted into the two arms of the experiment. In this data, half of users were supposed to be shown the old version of the site and half were supposed to see the new version.

`Pageviews` tells you how many unique users visited the welcome site we are experimenting on. `Pageviews` is what is sometimes called an "invariant" or "guardrail" variable, meaning that it shouldn't vary across treatment arms—after all, people have to visit the site before they get a chance to see the treatment, so there's no way that being assigned to treatment or control should affect the number of pageviews assigned to each group.

"Invariant" variables are also an example of what are known as a "pre-treatment" variable, because pageviews are determined before users are manipulated in any way. That makes it analogous to gender or age in experiments where you have demographic data—a person's age and gender are determined before they experience any manipulations, so the value of any pre-treatment attributes should be the same across the two arms of our experiment. This is what we've previously called "checking for balance," If pre-treatment attributes aren't balanced, then we may worry our attempt to randomly assign people to different groups failed.  Kohavi, Tang and Xu call this a "trust-based guardrail metric" because it helps us determine if we should trust our data.

To test the quality of the randomization, calculate the average number of pageviews for the treated group and for the control group. Do they look similar?


In [36]:
Dataset

,Date,Pageviews,Clicks,Enrollments,Payments,group,Enrollment Rate,PaymentAmongClicked,Enroll_Pay
34,"Fri, Nov 14",9192,735,NaN,NaN,control,NaN,NaN,NaN
34,"Fri, Nov 14",9304,770,NaN,NaN,treatment,NaN,NaN,NaN
27,"Fri, Nov 7",9424,781,NaN,NaN,control,NaN,NaN,NaN
27,"Fri, Nov 7",9272,767,NaN,NaN,treatment,NaN,NaN,NaN
6,"Fri, Oct 17",9088,780,127.0,44.0,treatment,0.162821,0.056410,0.106410
...,...,...,...,...,...,...,...,...,...
4,"Wed, Oct 15",9793,832,140.0,94.0,treatment,0.168269,0.112981,0.055288
11,"Wed, Oct 22",9947,838,162.0,92.0,control,0.193317,0.109785,0.083532
11,"Wed, Oct 22",9737,801,128.0,70.0,treatment,0.159800,0.087391,0.072409
18,"Wed, Oct 29",9262,727,201.0,96.0,treatment,0.276479,0.132050,0.144429


In [37]:
control_average_pageviews = Dataset[Dataset["group"] == "control"]["Pageviews"].mean()
treatment_average_pageviews = Dataset[Dataset["group"] == "treatment"][
    "Pageviews"
].mean()

print(f"The control group pageview average is {control_average_pageviews}")
print(f"The treatment group pageview average is {treatment_average_pageviews}")
print(
    "The pageview for control group and treatment group is similar with the average of control group being higher than that of the treatment gorup."
)

The control group pageview average is 9339.0
The treatment group pageview average is 9315.135135135135
The pageview for control group and treatment group is similar with the average of control group being higher than that of the treatment gorup.


### Exercise 7

"Similar" is a tricky concept -- obviously, we expect *some* differences across groups since users were *randomly* divided across treatment arms. The question is whether the differences between groups are larger than we'd expect to emerge given our random assignment process. To evaluate this, let's use a `ttest` to test the statistical significance of the differences we see. 

**Note**: Remember that scipy functions don't accept `pandas` objects, so you use a scipy function, you have to pass the numpy vectors underlying your data with the `.values` operator (e.g. `df.my_column.values`). 

Does the difference in `pageviews` look statistically significant?

Store the resulting p-value in `ex7_ttest_pvalue` **rounded to four decimal places.**

In [38]:
from scipy.stats import ttest_ind

pageviews_control = Dataset[Dataset["group"] == "control"]["Pageviews"].values
pageviews_treatment = Dataset[Dataset["group"] == "treatment"]["Pageviews"].values

t_statistic, p_value = ttest_ind(pageviews_treatment, pageviews_control)

ex7_ttest_pvalue = round(p_value, 4)
results["ex7_ttest_pvalue"] = ex7_ttest_pvalue

> A p_value of 0.8877 means we fail to reject the null hypothesis that there is no difference between the means in the control group and treatment group. 
From the data observed, we cannot conclude there is a difference between-group  of the pageviews.

### Exercise 8

`Pageviews` is not the only "pre-treatment" variable in this data we can use to evaluate balance/use as a guardrail metric. What other measure is pre-treatment? Review the description of the experiment if you're not sure.

We can also use clicks as a guardrail pre-treatment variable as the control group of clicks should be similar as that of the treatment group.

In [39]:
control_average_clicks = Dataset[Dataset["group"] == "control"]["Clicks"].mean()
treatment_average_clicks = Dataset[Dataset["group"] == "treatment"]["Clicks"].mean()


print(f"The control group pageview average is {control_average_clicks}")
print(f"The treatment group pageview average is {treatment_average_clicks}")
print(
    "The clicks for control group and treatment group is similar with the average of control group being slightly higher than that of the treatment gorup."
)

The control group pageview average is 766.972972972973
The treatment group pageview average is 765.5405405405405
The clicks for control group and treatment group is similar with the average of control group being slightly higher than that of the treatment gorup.


### Exercise 9

Check if the other pre-treatment variable is also balanced. Store the p-value of your test of difference in `results` under the key `"ex9_ttest_pvalue_clicks"` **rounded to four decimal places.**

In [40]:
clicks_control = Dataset[Dataset["group"] == "control"]["Clicks"].values
clicks_treatment = Dataset[Dataset["group"] == "treatment"]["Clicks"].values

t_statistic, p_value = ttest_ind(clicks_treatment, clicks_control)

ex9_ttest_pvalue_clicks = round(p_value, 4)
results["ex9_ttest_pvalue_clicks"] = ex9_ttest_pvalue_clicks
results

{'ex4_avg_oec': 0.0941,
 'ex5_avg_guardrail': 0.1158,
 'ex7_ttest_pvalue': 0.8877,
 'ex9_ttest_pvalue_clicks': 0.9264}

> A p_value of 0.9264 means we fail to reject the null hypothesis that there is no difference between the means in the control group and treatment group. 
Therefore we cannot conclude that there no difference in the mean number of clicks between the treatment and control groups

## Estimating the Effect of Experiment

### Exercise 10

Now that we've validated our randomization, our next task is to estimate our treatment effect. First, though, there's an issue with your data you've been able to largely ignore until now, but which you should get a grip on before estimating your treatment effect — can you tell what it is and what you should do about it?

Store the number of observations in your data *after* you've addressed this in `ex10_num_obs` (this is mostly meant as a way to sanity check your answer with autograder).

In [41]:
dataset_without_nas = Dataset.dropna()
ex10_num_obs = dataset_without_nas.shape[0]
results["ex10_num_obs"] = ex10_num_obs

In [42]:
results

{'ex4_avg_oec': 0.0941,
 'ex5_avg_guardrail': 0.1158,
 'ex7_ttest_pvalue': 0.8877,
 'ex9_ttest_pvalue_clicks': 0.9264,
 'ex10_num_obs': 46}


### Exercise 11

Now that we've established we have good balance (meaning we think randomization was likely successful), we can evaluate the effects of the experiment. Test whether the OEC and the metric you *don't* want affected have different average values in the control group and treatment group. 

Because we've randomized, this is a consistent estimate of the Average Treatment Effect of Udacity's website change.

Calculate the difference in means in your OEC and guardrail metrics using a simple t-test. Store the resulting effect estimates in `ex11_oec_ate` and `ex11_guard_ate` and p-values in `ex11_oec_pvalue` and `ex11_guard_pvalue`. **Please round all answers to 4 decimal places.** Report your ATE in *percentage points*, where `1` denotes 1 percentage point.


In [43]:
oec_group = dataset_without_nas.groupby(["group"], as_index=False)["Enroll_Pay"].mean()

oec_group.rename(
    {"Enroll_Pay": "Enroll And Payment Rate Difference"}, axis=1, inplace=True
)

oec_group

,group,Enroll And Payment Rate Difference
0,control,0.102082
1,treatment,0.086194


In [53]:
oec_control = dataset_without_nas[dataset_without_nas["group"] == "control"][
    "Enroll_Pay"
].values
oec_treatment = dataset_without_nas[dataset_without_nas["group"] == "treatment"][
    "Enroll_Pay"
].values


t_stats_11_oec, t_p_value_11_oec = ttest_ind(oec_treatment, oec_control)

oec_ate = (
    oec_group.loc[1]["Enroll And Payment Rate Difference"]
    - oec_group.loc[0]["Enroll And Payment Rate Difference"]
)

results["ex11_oec_ate"] = round(abs(oec_ate * 100), 4)
results["ex11_oec_pvalue"] = round(t_p_value_11_oec, 4)
results

{'ex4_avg_oec': 0.0941,
 'ex5_avg_guardrail': 0.1158,
 'ex7_ttest_pvalue': 0.8877,
 'ex9_ttest_pvalue_clicks': 0.9264,
 'ex10_num_obs': 46,
 'ex11_oec_ate': 1.5888,
 'ex11_oec_pvalue': 0.1319}

>The negative value indicates that, on average, enrollment rates in the treatment group were approximately 21.8355 percentage points lower than in the control group.
This could also be implied as the new feature proposed in the treatment group meant that they were 21.8355 less likely to enroll in the courses as compared to the control group.


> With a p-value of `0.1308` we fail to reject the null hypothesis that there is no difference in the means between the Enrollment rate in the Control and the Treatment group. So we do not have enough evidence to conclude there is no difference in the means in the OEC

In [22]:
guard_group

,group,PaymentAmongClicked
0,control,0.118269
1,treatment,0.113373


In [55]:
guard_control = dataset_without_nas[dataset_without_nas["group"] == "control"][
    "PaymentAmongClicked"
].values
guard_treatment = dataset_without_nas[dataset_without_nas["group"] == "treatment"][
    "PaymentAmongClicked"
].values


t_stats_11_guard, t_p_value_11_guard = ttest_ind(guard_treatment, guard_control)

guard_ate = (
    guard_group.loc[1]["PaymentAmongClicked"]
    - guard_group.loc[0]["PaymentAmongClicked"]
) * 100

results["ex11_guard_ate"] = round(abs(guard_ate), 4)
results["ex11_guard_pvalue"] = round(t_p_value_11_guard, 4)
results

{'ex4_avg_oec': 0.0941,
 'ex5_avg_guardrail': 0.1158,
 'ex7_ttest_pvalue': 0.8877,
 'ex9_ttest_pvalue_clicks': 0.9264,
 'ex10_num_obs': 46,
 'ex11_oec_ate': 1.5888,
 'ex11_oec_pvalue': 0.1319,
 'ex11_guard_ate': 0.4897,
 'ex11_guard_pvalue': 0.5928}

> With a p-value of `0.5928` we fail to reject the null hypothesis that there is no difference in the means between the guard rail metrics in the Control and the Treatment group. So we do not have enough evidence to conclude there is no difference in the means in the OEC

### Exercise 12

Do you feel that Udacity achieved their goal? Did their intervention cause them any problems? If they asked you "What would happen if we rolled this out to everyone?" what would you say?

As you answer this question, a small additional question: up until this point you've (presumably) been reporting the default p-values from the tools you are using. These, as you may recall from stats 101, are two-tailed p-values. Do those seem appropriate for your OEC?


**Based on the result of t-test on** the OEC, or **daily enrollment rate among those clicked** on *start free trial*, we **cannot conclude that there is a treatment effect**. **Because we are interested in whether the treatment would decrease OEC, a one-tail t-test would be more appropriate**.

### Exercise 13

One of the magic things about experiments is that all you have to do is compare averages to get an average treatment effect. However, you *can* do other things to try and increase the statistical power of your experiments, like add controls in a linear regression model. 

As you likely know, a bivariate regression is exactly equivalent to a t-test, so let's start by re-estimating the effect of treatment on your OEC using a linear regression. Can you replicate the results from your t-test? They shouldn't just be close—they should be numerically equivalent (i.e. exactly the same to the limits of floating point number precision). 

In [57]:
dataset_without_nas.rename(columns={"Enrollment Rate": "EnrollmentRate"}, inplace=True)

In [59]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# fit regression
model = ols(formula="Enroll_Pay ~ group", data=dataset_without_nas).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Enroll_Pay   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     2.356
Date:                Tue, 19 Mar 2024   Prob (F-statistic):              0.132
Time:                        00:48:25   Log-Likelihood:                 89.832
No. Observations:                  46   AIC:                            -175.7
Df Residuals:                      44   BIC:                            -172.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.1021      0.007     13.948      0.000       0.087       0.117
group[T.treatment]    -0.0159      0.010     -1.535      0.132      -0.037       0.005
==============================================================================
Omnibus:                       14.160   Durbin-Watson:                   1.481
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               15.205
Skew:                           1.227   Prob(JB):                     0.000499
Kurtosis:                       4.383   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [60]:
# check the p-values obtained with the models
assert round(model.pvalues[1], 4) == results["ex11_oec_pvalue"]

### Exercise 14

Now add indicator variables for the date of each observation. Do the standard errors on your `treatment` variable change? If so, in what direction?

Store your new standard error in `ex14_se_treatment`. Round your answer to 4 decimal places.

You should have found that your standard errors decreased by about 30\%—this is why, although just comparing means *works*, if you have additional variables adding them to your analysis can be helpful (all the usual rules for model specification apply — for example, you still want to be careful about overfitting, which one could argue is maybe part of what's happening here). 

In many other cases, the effect of adding controls is likely to be larger — the date indicators we added to our data are perfectly balanced between treatment and control, so we aren't adding a lot of data to the model by adding them as variables. They're accounting for some day-to-day variation (presumably in the types of people coming to the site), but they aren't controlling for any residual baseline differences the way a control like "gender" or "age" might (since those kind of individual-level attributes will never be perfectly balanced across treatment and control). 

In [61]:
# fit regression

model1 = ols(formula="Enroll_Pay ~ group + C(Date)", data=dataset_without_nas).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Enroll_Pay   R-squared:                       0.806
Model:                            OLS   Adj. R-squared:                  0.602
Method:                 Least Squares   F-statistic:                     3.962
Date:                Tue, 19 Mar 2024   Prob (F-statistic):           0.000978
Time:                        00:50:06   Log-Likelihood:                 126.29
No. Observations:                  46   AIC:                            -204.6
Df Residuals:                      22   BIC:                            -160.7
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0.1079      0.016      6.651      0.000       0.074       0.142
group[T.treatment]        -0.0159      0.007     -2.398      0.025      -0.030      -0.002
C(Date)[T.Fri, Oct 24]     0.0445      0.022      1.983      0.060      -0.002       0.091
C(Date)[T.Fri, Oct 31]    -0.0074      0.022     -0.331      0.744      -0.054       0.039
C(Date)[T.Mon, Oct 13]    -0.0231      0.022     -1.026      0.316      -0.070       0.024
C(Date)[T.Mon, Oct 20]    -0.0285      0.022     -1.270      0.217      -0.075       0.018
C(Date)[T.Mon, Oct 27]     0.0328      0.022      1.458      0.159      -0.014       0.079
C(Date)[T.Sat, Nov 1]     -0.0235      0.022     -1.047      0.306      -0.070       0.023
C(Date)[T.Sat, Oct 11]    -0.0017      0.022     -0.074      0.941      -0.048       0.045
C(Date)[T.Sat, Oct 18]    -0.0438      0.022     -1.950      0.064      -0.090       0.003
C(Date)[T.Sat, Oct 25]    -0.0309      0.022     -1.375      0.183      -0.077       0.016
C(Date)[T.Sun, Nov 2]      0.0549      0.022      2.441      0.023       0.008       0.101
C(Date)[T.Sun, Oct 12]    -0.0347      0.022     -1.542      0.137      -0.081       0.012
C(Date)[T.Sun, Oct 19]    -0.0178      0.022     -0.791      0.437      -0.064       0.029
C(Date)[T.Sun, Oct 26]    -0.0222      0.022     -0.989      0.333      -0.069       0.024
C(Date)[T.Thu, Oct 16]    -0.0228      0.022     -1.016      0.321      -0.069       0.024
C(Date)[T.Thu, Oct 23]    -0.0046      0.022     -0.203      0.841      -0.051       0.042
C(Date)[T.Thu, Oct 30]     0.0588      0.022      2.618      0.016       0.012       0.105
C(Date)[T.Tue, Oct 14]    -0.0417      0.022     -1.855      0.077      -0.088       0.005
C(Date)[T.Tue, Oct 21]    -0.0059      0.022     -0.260      0.797      -0.052       0.041
C(Date)[T.Tue, Oct 28]    -0.0287      0.022     -1.276      0.215      -0.075       0.018
C(Date)[T.Wed, Oct 15]    -0.0132      0.022     -0.588      0.562      -0.060       0.033
C(Date)[T.Wed, Oct 22]    -0.0220      0.022     -0.980      0.338      -0.069       0.025
C(Date)[T.Wed, Oct 29]     0.0466      0.022      2.076      0.050    4.77e-05       0.093
==============================================================================
Omnibus:                        3.871   Durbin-Watson:                   2.907
Prob(Omnibus):                  0.144   Jarque-Bera (JB):                3.826
Skew:                           0.000   Prob(JB):                        0.148
Kurtosis:                       4.413   Cond. No.                         27.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [64]:
std_er_dif = (model1.bse[1] - model.bse[1]) / model.bse[1]

print(
    f"After adding date to the regression model,\
 the standard error changed {std_er_dif:.4f} percent."
)

After adding date to the regression model, the standard error changed -0.3599 percent.


In [67]:
model1.bse[1]

0.0066253765225568785

In [65]:
results["ex14_se_treatment"] = round(model1.bse[1], 4)
results

{'ex4_avg_oec': 0.0941,
 'ex5_avg_guardrail': 0.1158,
 'ex7_ttest_pvalue': 0.8877,
 'ex9_ttest_pvalue_clicks': 0.9264,
 'ex10_num_obs': 46,
 'ex11_oec_ate': 1.5888,
 'ex11_oec_pvalue': 0.1319,
 'ex11_guard_ate': 0.4897,
 'ex11_guard_pvalue': 0.5928,
 'ex14_se_treatment': 0.0066}


After adding date to the regression model (controling for date), the standard error **decreased from .010 to .007** (~ 36 percent).

In [68]:
assert set(results.keys()) == {
    "ex4_avg_oec",
    "ex5_avg_guardrail",
    "ex7_ttest_pvalue",
    "ex9_ttest_pvalue_clicks",
    "ex10_num_obs",
    "ex11_guard_ate",
    "ex11_guard_pvalue",
    "ex11_oec_ate",
    "ex11_oec_pvalue",
    "ex14_se_treatment",
}

### Exercise 15

Does this result have any impact on the recommendations you would offer Udacity?



**By controlling for daily variance using the regression model** (adding the *Date* variable as a predictor), the **standard error of treatment effect decreased and the p-value dropped to below .05**, **indicating that controlling for daily variance, the treatment effect observed was unlikely to be due to sampling**. With this, we could conclude that **the treatment of adding a prompt making sure that the user can devote sufficient amount of time per week** ***does have an impact*** **on the metric of interest**: **it decreases the share of users clicking on the** *start free trial* **button who enrolled for free trial minus users who continued to pay after the trial**, reducing the coaching resources allocated to people who do not have the time to really engage with the materials during the trial period.